In [143]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd


'''
<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Wed 16</h2>
<span data-testid="TemperatureValue" class="DetailsSummary--highTempValue--3x6cL">57°</span>
<span class="DailyContent--daypartDate--3MM0J">Wed 09</span>
<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Thu 10</h2>
<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Fri 11</h2>
<span data-testid="TemperatureValue" class="DailyContent--temp--_8DL5">56°</span>
<p data-testid="wxPhrase" class="DailyContent--narrative--3AcXd">Partly cloudy this evening with more clouds for overnight. Low 44F. Winds light and variable.</p>
'''


'\n<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Wed 16</h2>\n<span data-testid="TemperatureValue" class="DetailsSummary--highTempValue--3x6cL">57°</span>\n<span class="DailyContent--daypartDate--3MM0J">Wed 09</span>\n<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Thu 10</h2>\n<h2 data-testid="daypartName" class="DetailsSummary--daypartName--1Mebr">Fri 11</h2>\n<span data-testid="TemperatureValue" class="DailyContent--temp--_8DL5">56°</span>\n<p data-testid="wxPhrase" class="DailyContent--narrative--3AcXd">Partly cloudy this evening with more clouds for overnight. Low 44F. Winds light and variable.</p>\n'

In [163]:
page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")

soup = BeautifulSoup(page.content, 'html.parser')

tendays = soup.find("div", {"class" : "DailyForecast--DisclosureList--350ZO"})

weatherData = {}
dayIndex = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
label = ['Temperature:', 'Conditions:', 'Item Name:']

for i in dayIndex:
    day = tendays.find("div", {"id" : f"detailIndex{i}"})
    item = f"detailIndex{i}"
    date = list(day.children)[0].get_text() + '.12'
    temp = list(day.children)[1].get_text()
    cond = list(day.children)[2].get_text()
    weatherData[date] = [temp, cond, item]

df = pd.DataFrame(weatherData, index=label)
print(df)

# :D 

                 Thu 10.12      Fri 11.12     Sat 12.12     Sun 13.12  \
Temperature:       56°/44°        55°/45°       56°/46°       57°/47°   
Conditions:   Mostly Sunny  Partly Cloudy       Showers          Rain   
Item Name:    detailIndex1   detailIndex2  detailIndex3  detailIndex4   

                  Mon 14.12      Tue 15.12     Wed 16.12     Thu 17.12  \
Temperature:        57°/44°        56°/44°       57°/45°       57°/44°   
Conditions:   Partly Cloudy  Partly Cloudy    PM Showers       Showers   
Item Name:     detailIndex5   detailIndex6  detailIndex7  detailIndex8   

                 Fri 18.12      Sat 19.12  
Temperature:       57°/44°        59°/44°  
Conditions:     AM Showers     AM Showers  
Item Name:    detailIndex9  detailIndex10  
